In [1]:
include("categories.jl")

Begin...
Checking associativity...
("h", "g", "f")
("id2", "g", "f")
("g", "id1", "f")
("id1", "id1", "f")
("k", "id1", "f")
("id3", "k", "f")
("id3", "id3", "id3")
("id3", "id3", "h")
("id3", "h", "i")
("h", "id2", "i")
("id2", "id2", "i")
("id3", "h", "g")
("h", "id2", "g")
("id2", "id2", "g")
("h", "g", "id1")
("id2", "g", "id1")
("g", "id1", "id1")
("id1", "id1", "id1")
("k", "id1", "id1")
("id3", "k", "id1")
("id3", "id3", "k")
("g", "f", "id0")
("id1", "f", "id0")
("k", "f", "id0")
("h", "i", "id0")
("id2", "i", "id0")
("f", "id0", "id0")
("i", "id0", "id0")
("id0", "id0", "id0")
("m", "id0", "id0")
("n", "id0", "id0")
("id3", "m", "id0")
("id3", "n", "id0")
("id3", "id3", "m")
("id3", "h", "id2")
("h", "id2", "id2")
("id2", "id2", "id2")
("id3", "id3", "n")
Checking associativity...
("h", "g", "f")
("id2", "g", "f")
("g", "id1", "f")
("id1", "id1", "f")
("k", "id1", "f")
("id3", "k", "f")
("id3", "id3", "id3")
("id3", "id3", "h")
("id3", "h", "i")
("h", "id2", "i")
("id2", "id2"

In [5]:
keys(cat1.src) == keys(cat1.target)

true

In [14]:
#using NBInclude
#nbexport("test.jl", "categories.ipynb")